In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression as LR
np.random.seed(42)

In [2]:
def score(X, W):
    scr = np.dot(X, W)
    return scr

In [3]:
def sigmoid(z):
    a = 1.0 / (1.0 + np.exp(-z))
    return a

In [4]:
def sigmoid_derive(a):
    d = a * (1.0 - a)
    return d

In [5]:
def f_log_loss(y_true, p):
    log_loss = np.mean(-y_true * np.log(p) - (1.0 - y_true) * np.log(1.0 - p))
    return log_loss

In [6]:
n_features = 2
x_size = 10000
X = np.random.sample((x_size, n_features+1))
X[:, 0] = 1
X

array([[ 1.        ,  0.95071431,  0.73199394],
       [ 1.        ,  0.15601864,  0.15599452],
       [ 1.        ,  0.86617615,  0.60111501],
       ..., 
       [ 1.        ,  0.15825956,  0.12330421],
       [ 1.        ,  0.95227545,  0.74782421],
       [ 1.        ,  0.40100484,  0.25739798]])

In [7]:
W_true = np.array([
    [-0.4],
    [0.4],
    [0.5]
])

In [8]:
y_true = np.round(sigmoid(score(X, W_true)))
y_true.mean()

0.59340000000000004

In [9]:
#! Add noise
X[:, 1:] += np.random.sample(X.shape)[:, 1:] / 1.5
X

array([[ 1.        ,  1.25690928,  1.37499296],
       [ 1.        ,  0.54792292,  0.6228012 ],
       [ 1.        ,  1.13749017,  1.05906295],
       ..., 
       [ 1.        ,  0.3589041 ,  0.77028208],
       [ 1.        ,  1.59636927,  0.99665085],
       [ 1.        ,  0.67257986,  0.60595357]])

#### #Newton method

In [10]:
#initialize W
W_init = np.random.sample(W_true.shape) / 10
print(W_init, "\n\n",W_true)

[[ 0.07415552]
 [ 0.08811019]
 [ 0.04631799]] 

 [[-0.4]
 [ 0.4]
 [ 0.5]]


In [121]:
lam = 200 # L2 regularization 
alpha = 0.003
iter1 = 100
W_prev, W = np.zeros(W.shape), np.copy(W_init)
for i in range(iter1):
    #probability using current W
    p = sigmoid(score(X, W))
    p_derive = sigmoid_derive(p)
    #Hessian
    A = np.copy(X)
    B = np.identity(x_size) * p_derive
    H = np.dot(np.dot(A.T, B), A)
    H_inv = np.linalg.inv(H + lam*np.eye(W.shape[0]))  # 3x3
    #gradient
    grad = np.dot(A.T, (p - y_true)) #3x1
    #print(H_inv.shape, grad.shape)
    #step = np.linalg.lstsq(Hf.T, grad.T)[0]
    #print('step.shape: {}'.format(step.shape))
    step = np.dot(H_inv, grad)
    #new W
    W -= alpha * step
    #gini
    auc = roc_auc_score(y_true, p)
    gini = 200 * auc - 100
    log_loss = f_log_loss(y_true, p)
    if i % (iter1 // 10) == 0:
        print('{}: gini = {}, log_loss = {}'.format(i, gini, log_loss))
        print(W)
        #for i in range(10):
            #print(int(y_true[i, 0]), p[i, 0])
        print('-' * 20)
    
    if np.max(np.abs(W - W_prev)) > 1e-8:
        W_prev = np.copy(W)
        continue
    else:
        print('The weight difference is small. Process stopped.')
        break

0: gini = 15.859158524045071, log_loss = 0.6894635536989396
[[-0.02916719]
 [ 0.09748021]
 [-0.04076173]]
--------------------
10: gini = 45.399042873314045, log_loss = 0.6826802700833441
[[-0.07065072]
 [ 0.12211186]
 [ 0.00127095]]
--------------------
20: gini = 62.00111852530114, log_loss = 0.6761312770221675
[[-0.11179831]
 [ 0.14644188]
 [ 0.04268418]]
--------------------
30: gini = 70.46284051926023, log_loss = 0.6698045480015583
[[-0.15261871]
 [ 0.17048158]
 [ 0.08350094]]
--------------------
40: gini = 74.89485579713286, log_loss = 0.6636888823102695
[[-0.19312003]
 [ 0.19424153]
 [ 0.12374273]]
--------------------
50: gini = 77.33235869438258, log_loss = 0.6577738357960654
[[-0.23330985]
 [ 0.21773159]
 [ 0.16342962]]
--------------------
60: gini = 78.7625906787232, log_loss = 0.652049658288886
[[-0.27319523]
 [ 0.24096098]
 [ 0.2025804 ]]
--------------------
70: gini = 79.6342910747762, log_loss = 0.6465072369744744
[[-0.31278273]
 [ 0.26393833]
 [ 0.24121266]]
-------

In [114]:
lr = LR()
lr.fit(X, y_true.ravel())
pred2 = lr.predict_proba(X)[:, 1]
print(200.0 * roc_auc_score(y_true.ravel(), pred2.ravel()) - 100)
lr.coef_

81.4845681889


array([[-3.91208425,  4.40717924,  5.91972339]])